# Pipeline Final - Classificação de Spam para Produção

## Objetivo

Treinar modelo FINAL para produção:
- Usar 100% dos dados (sem train/test split)
- Validar com cross-validation
- Exportar artefatos completos para API

## Configuração Final

Baseado nos notebooks anteriores:
- **Modelo**: SVM (LinearSVC) otimizado do notebook 03
- **Features**: TF-IDF (5000 features)
- **Vectorizer**: TfidfVectorizer do notebook 01
- **Performance**: Métricas do notebook 03


In [4]:
import pandas as pd
import numpy as np
import joblib
import os
import time
from pathlib import Path
from datetime import datetime
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVC
from sklearn.preprocessing import LabelEncoder

print("OK - Bibliotecas carregadas")


OK - Bibliotecas carregadas


## Passo 1: Carregar Dados Completos

Carregar todos os dados (sem split) para treinar modelo final.


In [5]:
# Carregar dataset completo
data_path = Path('data/emails.csv')
df = pd.read_csv(data_path)

print(f"Dataset completo: {len(df):,} emails")
print(f"Colunas: {df.columns.tolist()}")

# Separar features e target
X_full = df['message']
y_full = df['label']

print(f"\nX_full: {len(X_full):,} mensagens")
print(f"y_full: {len(y_full):,} labels")
print(f"\nDistribuição de classes:")
print(y_full.value_counts())


Dataset completo: 83,448 emails
Colunas: ['label', 'message']

X_full: 83,448 mensagens
y_full: 83,448 labels

Distribuição de classes:
label
spam    43910
ham     39538
Name: count, dtype: int64


## Passo 2: Carregar Vetorizador e Modelo Otimizado

Carregar o vetorizador TF-IDF do notebook 01 e o modelo otimizado do notebook 03.


In [6]:
# Carregar vetorizador do notebook 01
vectorizer_path = Path('artifacts/tfidf_vectorizer.joblib')
if not vectorizer_path.exists():
    raise FileNotFoundError("Vetorizador não encontrado. Execute o notebook 01 primeiro.")

vectorizer = joblib.load(vectorizer_path)
print("Vetorizador TF-IDF carregado!")

# Carregar modelo otimizado do notebook 03
optimized_model_path = Path('artifacts/optimized_model.joblib')
if not optimized_model_path.exists():
    raise FileNotFoundError("Modelo otimizado não encontrado. Execute o notebook 03 primeiro.")

optimized_model = joblib.load(optimized_model_path)
print("Modelo otimizado carregado!")

# Carregar resultados da otimização
tuning_results_path = Path('artifacts/hyperparameter_tuning_results.joblib')
if tuning_results_path.exists():
    tuning_results = joblib.load(tuning_results_path)
    print(f"Modelo: {tuning_results.get('model_type', 'Unknown')}")
    print(f"F1-Score esperado: {tuning_results.get('test_f1', 0):.4f}")
else:
    tuning_results = {}
    print("Resultados da otimização não encontrados.")


Vetorizador TF-IDF carregado!
Modelo otimizado carregado!
Modelo: SVM (LinearSVC)
F1-Score esperado: 0.9872


## Passo 3: Treinar Modelo Final

Treinar modelo final com 100% dos dados usando os melhores hiperparâmetros encontrados.


In [7]:
# Vetorizar todos os dados
print("Vetorizando todos os dados...")
X_full_tfidf = vectorizer.transform(X_full)
print(f"Shape final: {X_full_tfidf.shape}")

# Treinar modelo final com todos os dados
print("\nTreinando modelo final com 100% dos dados...")
print("Isso pode levar alguns minutos...")

start_time = time.time()

# Criar novo modelo LinearSVC com os mesmos parâmetros do otimizado
final_model = LinearSVC(**optimized_model.get_params())

# LinearSVC precisa de labels numéricos
label_encoder = LabelEncoder()
y_full_encoded = label_encoder.fit_transform(y_full)
final_model.fit(X_full_tfidf, y_full_encoded)

print("Modelo LinearSVC treinado com labels numéricos")
print(f"Mapeamento: {dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))}")

training_time = time.time() - start_time

print(f"\nModelo final treinado em {training_time:.2f}s")
print(f"Total de amostras de treino: {len(y_full):,}")


Vetorizando todos os dados...
Shape final: (83448, 5000)

Treinando modelo final com 100% dos dados...
Isso pode levar alguns minutos...
Modelo LinearSVC treinado com labels numéricos
Mapeamento: {'ham': np.int64(0), 'spam': np.int64(1)}

Modelo final treinado em 1.45s
Total de amostras de treino: 83,448


## Passo 4: Validação com Cross-Validation

Validar modelo final usando cross-validation para garantir robustez.


In [8]:
# Cross-validation com F1-Score
print("Executando 5-fold cross-validation...")
print("Isso pode levar alguns minutos...\n")

# Cross-validation com labels numéricos (LinearSVC)
cv_scores = cross_val_score(
    final_model,
    X_full_tfidf,
    y_full_encoded,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

print("Resultados do Cross-Validation:")
print(f"  F1-Score médio: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"  Scores individuais: {cv_scores}")
print(f"  Min: {cv_scores.min():.4f}")
print(f"  Max: {cv_scores.max():.4f}")


Executando 5-fold cross-validation...
Isso pode levar alguns minutos...

Resultados do Cross-Validation:
  F1-Score médio: 0.9856 (+/- 0.0014)
  Scores individuais: [0.98616937 0.98634019 0.9847756  0.986056   0.98477446]
  Min: 0.9848
  Max: 0.9863


## Passo 5: Exportar Artefatos para Produção

Salvar modelo final, vetorizador e metadados para uso na API.


In [9]:
# Criar diretório para modelos finais
models_dir = Path('artifacts')
models_dir.mkdir(exist_ok=True)

# 1. Salvar modelo final
final_model_path = models_dir / 'best_model_temp.joblib'
joblib.dump(final_model, final_model_path)
print(f"✓ Modelo final salvo: {final_model_path}")

# 2. Salvar vetorizador (já existe, mas vamos garantir)
vectorizer_final_path = models_dir / 'tfidf_vectorizer.joblib'
joblib.dump(vectorizer, vectorizer_final_path)
print(f"✓ Vetorizador salvo: {vectorizer_final_path}")

# 3. Salvar label_encoder (necessário para LinearSVC)
label_encoder_path = models_dir / 'label_encoder.joblib'
joblib.dump(label_encoder, label_encoder_path)
print(f"✓ Label encoder salvo: {label_encoder_path}")

# 4. Salvar metadados
metadata = {
    'model_type': 'LinearSVC',
    'model_params': final_model.get_params(),
    'vectorizer_type': 'TfidfVectorizer',
    'vectorizer_params': vectorizer.get_params(),
    'training_samples': len(y_full),
    'cv_f1_mean': float(cv_scores.mean()),
    'cv_f1_std': float(cv_scores.std()),
    'trained_date': datetime.now().isoformat(),
    'features_count': X_full_tfidf.shape[1],
    'vocabulary_size': len(vectorizer.vocabulary_),
    'label_encoder': True,
    'label_mapping': dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
}

# Adicionar métricas do notebook 03 se disponíveis
if tuning_results:
    metadata.update({
        'optimization_f1': tuning_results.get('test_f1'),
        'optimization_accuracy': tuning_results.get('test_accuracy'),
        'optimization_precision': tuning_results.get('test_precision'),
        'optimization_recall': tuning_results.get('test_recall')
    })

metadata_path = models_dir / 'metadata.joblib'
joblib.dump(metadata, metadata_path)
print(f"✓ Metadados salvos: {metadata_path}")

print("\n" + "="*80)
print("ARTEFATOS EXPORTADOS PARA PRODUÇÃO")
print("="*80)
print("1. best_model_temp.joblib (modelo treinado)")
print("2. tfidf_vectorizer.joblib (vetorizador TF-IDF)")
print("3. label_encoder.joblib (encoder de labels)")
print("4. metadata.joblib (informações do modelo)")


✓ Modelo final salvo: artifacts/best_model_temp.joblib
✓ Vetorizador salvo: artifacts/tfidf_vectorizer.joblib
✓ Label encoder salvo: artifacts/label_encoder.joblib
✓ Metadados salvos: artifacts/metadata.joblib

ARTEFATOS EXPORTADOS PARA PRODUÇÃO
1. best_model_temp.joblib (modelo treinado)
2. tfidf_vectorizer.joblib (vetorizador TF-IDF)
3. label_encoder.joblib (encoder de labels)
4. metadata.joblib (informações do modelo)


## Conclusão do Pipeline Final

### Modelo Final Treinado

O modelo final foi treinado com 100% dos dados e validado com cross-validation.

### Artefatos Exportados

Todos os artefatos foram salvos em `artifacts/` e estão prontos para deploy na API:
- `best_model_temp.joblib` - Modelo treinado
- `tfidf_vectorizer.joblib` - Vetorizador TF-IDF
- `metadata.joblib` - Metadados do modelo

### Próximos Passos

1. Executar `scripts/deploy_models.py` para copiar modelos para `api-service/models/`
2. Testar API localmente
3. Fazer deploy em produção
